In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2") 
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")  
openai_api_key = os.getenv("OPENAI_API_KEY") 

os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model="gpt-3.5-turbo")
vector_store_path = "./vector_store/"

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

pdf_path = "DatasetUU/UU_NO_1_2024.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

if os.path.exists(vector_store_path):
    vector_store = FAISS.load_local(vector_store_path, embeddings=embeddings, allow_dangerous_deserialization=True)
    _ = vector_store.add_documents(documents=all_splits)
else:
    vector_store = InMemoryVectorStore(embeddings)
    vector_store = FAISS.from_documents(all_splits, embedding=embeddings)
    vector_store.save_local("./vector_store/")

prompt = hub.pull("rlm/rag-prompt")

C:\Users\Frederick\AppData\Local\Temp\ipykernel_10080\3358133716.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [5]:
from typing_extensions import List, TypedDict
from langgraph.graph import START, StateGraph
from langchain_core.documents import Document

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [6]:
response = graph.invoke({"question": "Saya tidak sengaja menemukan data privasi orang lain apakah saya akan terkena pidana, namun tidak menyebarkannya?"})
print(response["answer"])

Jika Anda tidak sengaja menemukan data privasi orang lain dan tidak menyebarkannya, Anda tidak akan terkena pidana berdasarkan informasi yang diberikan. Namun, penuntutan pidana hanya dapat dilakukan atas aduan jika informasi tersebut disebarkan tanpa seizin atau tanpa hak. Jadi, penting untuk menjaga kerahasiaan informasi pribadi orang lain untuk menghindari masalah hukum.
